In [1]:
from memory_profiler import memory_usage
import time
import numpy as np

# ExaFMM-T
import exafmm.laplace as laplace

# PyExaFMM
from fmm import Fmm
from fmm.kernel import laplace_p2p_serial, laplace_gradient
from fmm.surface import scale_surface

# Adaptoctree
import adaptoctree.morton as morton
import adaptoctree.tree as tree

In [2]:
# Compile test experiment to cache numba functions
e = Fmm('test')
e.run()
laplace_p2p_serial(e.sources, e.targets, e.source_densities)
laplace_gradient(e.sources, e.targets, e.source_densities)

array([[  505.32437622,  1662.06575545, -1036.23303044],
       [ -319.91075759,   370.52167043,  1357.60678574],
       [  344.01185189,   913.15129831,   112.52339329],
       ...,
       [ -155.78901703,  -214.95415479,  -778.77791716],
       [ -168.37581363,  -681.57519089,  -444.27740462],
       [ -190.25537075,  -121.56253011,  -868.31197325]])

In [3]:
# ! rm test.hdf5 && fmm generate-test-data -c test && fmm compute-operators -c test
# ! rm 10_6_r.hdf5 && fmm generate-test-data -c 10_6_r && fmm compute-operators -c 10_6_r
# ! rm 10_6_s.hdf5 && fmm generate-test-data -c 10_6_s && fmm compute-operators -c 10_6_s
# ! rm 100_6_r.hdf5 && fmm generate-test-data -c 100_6_r && fmm compute-operators -c 100_6_r
# ! rm 100_6_s.hdf5 && fmm generate-test-data -c 100_6_s && fmm compute-operators -c 100_6_s
# ! rm full_6_r.hdf5 && fmm generate-test-data -c full_6_r && fmm compute-operators -c full_6_r
# ! rm full_6_s.hdf5 && fmm generate-test-data -c full_6_s && fmm compute-operators -c full_6_s

# ! rm test.hdf5 && fmm generate-test-data -c test && fmm compute-operators -c test
# ! rm 10_10_r.hdf5 && fmm generate-test-data -c 10_10_r && fmm compute-operators -c 10_10_r
# ! rm 10_10_s.hdf5 && fmm generate-test-data -c 10_10_s && fmm compute-operators -c 10_10_s
# ! rm 100_10_r.hdf5 && fmm generate-test-data -c 100_10_r && fmm compute-operators -c 100_10_r
# ! rm 100_10_s.hdf5 && fmm generate-test-data -c 100_10_s && fmm compute-operators -c 100_10_s
# ! rm full_10_r.hdf5 && fmm generate-test-data -c full_10_r && fmm compute-operators -c full_10_r
# ! rm full_10_s.hdf5 && fmm generate-test-data -c full_10_s && fmm compute-operators -c full_10_s

In [4]:
# Discretisation order of check surface
pvec = [6]
kvec = [10, 100, 'full']


# Load experimental data into PyExaFMM
pyfmmvec = []

for p in pvec:
    for k in kvec:
        pyfmmvec.append(Fmm(f'{k}_{p}_r'))
        pyfmmvec.append(Fmm(f'{k}_{p}_s'))

In [7]:
# Load experimental data into ExaFMM-T
exafmmvec = []
exafmmtreevec = []

for e in pyfmmvec:
    # create a list of source instances
    sources = laplace.init_sources(e.sources, e.source_densities)

    # create a list of target instances
    targets = laplace.init_targets(e.targets)
    
    # Expansion order
    p = e.config['order_equivalent']
    k = e.config['target_rank']
    fmm = laplace.LaplaceFmm(p=p, ncrit=e.config['max_points'], filename=f'{p}_{k}.dat')
    exafmmvec.append(fmm)
    
    tree = laplace.setup(sources, targets, fmm)
    exafmmtreevec.append(tree)

# Runtimes

In [5]:
# Evaluate PyExaFMM experiments
nruns = 1
pytvec = [[] for _ in range(nruns)]
for i in range(nruns):
    for e in pyfmmvec:
        e.clear()
    
    for e in pyfmmvec:
        s = time.time()
        e.run()
        pytvec[i].append(time.time()-s)
        
pytvec = np.array(pytvec)

In [8]:
# Evaluate ExaFMM-T experiments
nruns = 1
extvec = [[] for _ in range(nruns)]
exfmmresvec = []
for i in range(nruns):
    for tree, fmm in list(zip(exafmmtreevec, exafmmvec)):
        laplace.clear_values(tree)

    for tree, fmm in list(zip(exafmmtreevec, exafmmvec)):
        
        s = time.time()
        res = laplace.evaluate(tree, fmm)
        extvec[i].append(time.time()-s)
        exfmmresvec.append(res)
extvec = np.array(extvec)

In [6]:
pytvec.mean(axis=0)

array([18.5699594 , 15.09127378, 26.8401165 , 15.83036327, 59.61733389,
       18.29960728])

In [45]:
pytvec.std(axis=0)

array([0., 0., 0., 0., 0., 0.])

In [13]:
extvec.mean(axis=0)[:2]

array([2.49864817, 1.95940399])

In [14]:
extvec.std(axis=0)[:2]

array([0., 0.])

In [15]:
pyfmmvec[0].config['target_rank']

10

In [16]:
pytvec/extvec

array([[ 7.43200249,  7.70197154, 13.67345524,  8.9870119 , 22.61878072,
        10.51803743]])

# Memory

In [3]:
# Discretisation order of check surface
pvec = [6]
kvec = [10, 100, 'full']


# Load experimental data into PyExaFMM
pyfmmvec = []

for p in pvec:
    for k in kvec:
        pyfmmvec.append(Fmm(f'{k}_{p}_r'))
        pyfmmvec.append(Fmm(f'{k}_{p}_s'))

In [ ]:
pymemvec = []

for e in pyfmmvec:
    pymemvec.append(max(memory_usage(e.run)))

In [ ]:
pymemvec

In [10]:
for tree, fmm in list(zip(exafmmtreevec, exafmmvec)):
    laplace.clear_values(tree)

exmemvec = []

for tree, fmm in list(zip(exafmmtreevec, exafmmvec)):
    def f(): laplace.evaluate(tree, fmm)
    exmemvec.append(max(memory_usage(f)))

NameError: name 'exafmmtreevec' is not defined

In [26]:
exmemvec

[5477.78515625,
 5388.95703125,
 4924.71875,
 5387.90625,
 5477.92578125,
 5388.70703125]

# Accuracy

In [9]:
# idx = 0
# dtype = str(pyfmmvec[idx].config['data_type'])
# print(dtype)

# direct = laplace_p2p_serial(
#     pyfmmvec[idx].sources, 
#     pyfmmvec[idx].targets, 
#     pyfmmvec[idx].source_densities
# )

# with open(f'direct_{dtype}.pkl', 'wb') as f:
#     np.save(f, direct)

In [10]:
with open('direct_sphere.pkl', 'rb') as f:
    direct_sphere = np.load(f)

In [11]:
with open('direct_random.pkl', 'rb') as f:
    direct_random = np.load(f)

In [12]:
random_pyexperiments = [pyfmmvec[0], pyfmmvec[2], pyfmmvec[4]]
random_pyerrors = []

for e in random_pyexperiments:
    
    error = np.mean(abs(e.target_potentials[:,0]-direct_random)/direct_random)
    random_pyerrors.append(error)
    
    
random_exexperiments = [exfmmresvec[0], exfmmresvec[2], exfmmresvec[4]]
random_exerrors = []

for e in random_exexperiments:
    
    error = np.mean(abs(e[:,0]-direct_random)/direct_random)
    random_exerrors.append(error)

In [13]:
sphere_pyexperiments = [pyfmmvec[1], pyfmmvec[3], pyfmmvec[5]]
sphere_pyerrors = []

for e in sphere_pyexperiments:
    
    error = np.mean(abs(e.target_potentials[:,0]-direct_sphere)/direct_sphere)
    sphere_pyerrors.append(error)
    
    
sphere_exexperiments = [exfmmresvec[1], exfmmresvec[3], exfmmresvec[5]]
sphere_exerrors = []

for e in sphere_exexperiments:
    
    error = np.mean(abs(e[:,0]-direct_sphere)/direct_sphere)
    sphere_exerrors.append(error)

In [14]:
print(random_exerrors)

[3.3060505505988473e-07, 2.862516966947806e-06, 3.3060505505988473e-07]


In [15]:
print(random_pyerrors)

[0.00014588679303668398, 3.4635811419415778e-06, 3.936711732894845e-07]


In [16]:
sphere_pyerrors

[0.0004635461330072312, 4.5839533142968973e-07, 4.6142721735511585e-07]

In [17]:
sphere_exerrors

[3.8241347309400043e-07, 3.8241347309400043e-07, 3.8241347309400043e-07]

In [21]:
sphere_pyexperiments[0].config['target_rank']

10